In [ ]:
X_Initialization = pd.DataFrame()
for i in I:
    x = np.random.normal(loc=mu[i],scale=sigma[i],size=n_0)
    x = pd.DataFrame(x).T
    X_Initialization = pd.concat([X_Initialization,x])
X_Initialization.index = I

In [ ]:
for i in I:
    x_i = X_Initialization.loc[i]
    for l in I:
        x_l = X_Initialization.loc[l]
        x_il = x_i-x_l
        s_il = np.var(x_il,ddof=1)
        S_squire.loc[i,l] = s_il
    s_ii = np.var(x_i,ddof=1)
    S_squire.loc[i,i] = s_ii
S_diagnoal = np.power(np.diagonal(S_squire),0.5)
X_bar = np.mean(X_Initialization,axis=1)

In [ ]:
#Paulson
I_Paulson = I.copy()
r = 0
X = pd.DataFrame(index=range(10))
while r <3000 and len(I_Paulson)>1 :
    r += 1
    X[r]=np.random.normal(loc=mu,scale = sigma)/S_diagnoal + X_bar*(1-1/S_diagnoal)
    X_bar_r = np.mean(X,axis=1)
    I_old = I_Paulson.copy()
    I_Paulson = list()
    for l in I_old:
        if X_bar_r[l] >= max(X_bar_r)-max(0,a/r-Lambda):
            I_Paulson.append(l)

In [ ]:
#KN
I_KN = I.copy()
r = 0
X = pd.DataFrame(index=range(10))
while r <10000 and len(I_KN)>1 :
    r += 1
    X[r]=np.random.normal(loc=mu,scale = sigma)
    X_bar_r = np.mean(X,axis=1)
    I_old = I_KN.copy()
    I_KN = list()
    for i in I_old:
        keep_i = True
        for l in I_old:
            W_il_r = max(0,
                         delta/(2*r)*(h_squire*S_squire.loc[i,l]/np.power(delta,2)-r))
            if not X_bar_r[i]>=X_bar_r[l]-W_il_r:
                keep_i = False
        if keep_i:
            I_KN.append(i)


In [ ]:
X_Initialization = pd.DataFrame()
for i in I:
    x = np.random.normal(loc=mu[i],scale=sigma[i],size=n_0)
    x = pd.DataFrame(x).T
    X_Initialization = pd.concat([X_Initialization,x])
X_Initialization.index = I
#计算样本均值与样本方差
for i in I:
    x_i = X_Initialization.loc[i]
    for l in I:
        x_l = X_Initialization.loc[l]
        x_il = x_i-x_l
        s_il = np.var(x_il,ddof=1)
        n_ij = np.floor(h_squire_pp*s_il/Lambda)+1
        S_squire.loc[i,l] = s_il
        N_ij.loc[i,l] = n_ij
    s_ii = np.var(x_i,ddof=1)
    S_squire.loc[i,i] = s_ii
S_diagnoal = np.power(np.diagonal(S_squire),0.5)
X_bar = np.mean(X_Initialization,axis=1)
N = np.max(N_ij,axis=1)

#Paulson算法（来自论文Speeding Up Paulson’s Procedure for Large-Scale Problem Using Parallel Computing）
I_PP = I.copy()
t_PP = 0
X_PP = pd.DataFrame(index=range(10))
while len(I_PP)>1 and t_PP<np.max(N) :
    t_PP += 1
    X_PP[t_PP]=np.random.normal(loc=mu,scale = sigma)
    X_bar_t = np.mean(X_PP,axis=1)
    I_old = I_PP.copy()
    I_PP = list()
    for i in I_old:
        keep_i = True
        for j in I_old:
            Z_ij_t = t_PP*(X_bar_t[i]-X_bar_t[j])
            if not Z_ij_t>=-h_squire_pp*S_squire.loc[i,j]+Lambda*t_PP:
                keep_i = False
        if keep_i:
            I_PP.append(i)

if len(I_PP)>1:
    I_PP = list(np.argmax(X_bar_t))
